In [25]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# input dataset to notebook

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/training/simpsons_dataset-training.tsv
/kaggle/input/testing/simpsons_dataset-testing.tsv
/kaggle/input/submission/simpsons_dataset-sample_submission.csv


In [26]:
import numpy as np 
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/training/simpsons_dataset-training.tsv
/kaggle/input/testing/simpsons_dataset-testing.tsv
/kaggle/input/submission/simpsons_dataset-sample_submission.csv


In [27]:
# import BERT tokenization
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

In [28]:
# reading training ,testing,and sample_submission dataset
train=pd.read_csv('/kaggle/input/training/simpsons_dataset-training.tsv',sep='\t')
test=pd.read_csv('/kaggle/input/testing/simpsons_dataset-testing.tsv',sep='\t')
submission= pd.read_csv('/kaggle/input/submission/simpsons_dataset-sample_submission.csv')

In [29]:
train.isnull().sum()

id          0
class       0
subclass    0
text        0
dtype: int64

removing missing values

In [30]:
train = train.dropna().reset_index(drop=True)
train.isnull().sum()

id          0
class       0
subclass    0
text        0
dtype: int64

removing stopwords

In [31]:
import spacy
nlp = spacy.load('en_core_web_sm', disable=['ner', 'parser']) 

def cleaning(doc):
    # Lemmatizes and removes stopwords
    # doc needs to be a spacy Doc object
    txt = [ token.lemma_ for token in doc if not token.is_stop]
    if len(txt) > 2:
        return ' '.join(txt)

Remove all non-alphabetic characters

In [32]:
brief_cleaning = (re.sub("[^A-Za-z']+", ' ', str(row)).lower() for row in train['text'])

In [33]:
from time import time
import re
t = time()
txt = [ cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size=5000, n_process=-1)]

print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))

Time to clean up everything: 1.13 mins


In [34]:
#install package needed
pip install tokenization

Note: you may need to restart the kernel to use updated packages.


In [35]:
#install package needed
pip install bert-for-tf2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 799.8 kB/s eta 0:00:000:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-py3-none-any.whl size=30535 sha256=2e98e02aa6a2a4b1e6851b7c4256ab4c75eb0dd270ead7d25c5eecd67e1d906a
  Stored in directory: /root/.cache/pip/wheels/47/b6/e5/8c76ec779f54bc5c2f1b57d2200bb9c77616da83873e8acb53
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19472 sha256=d574ed9fd7e7f427f1810729333177316bf71121387f1b7a2488d702f6cd10ff
  Stored in directory: /root/.cache/pip/wheels/0e/fc/d2/a44fff33af0f233d7def6e7de413006d57c10e10ad736fe8f5
  Created wheel for py-params: filename=py_params-0.10.2-py3-none-any.whl size=7911 sha256=84c12c934f89378267a3da5a0ab0147f43560b4da9aa0365a4a9ea89f28cea6e
  Stored in directory: /root/.cache/pip/wheels/e1/11/67/33cc51bbee127cb8fb2ba549cd29109b2f22da43ddf

In [36]:
from bert import bert_tokenization
import tokenization
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.utils import to_categorical
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [37]:
#lable characters 
label = preprocessing.LabelEncoder()
y = label.fit_transform(train['class'])
y = to_categorical(y)
print(y[:5])

[[1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]]


In [38]:
#creating a bert embedding layer by importing the BERT model from hub.KerasLayer
m_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2'
bert_layer = hub.KerasLayer(m_url, trainable=True)

2022-10-26 02:50:53.864532: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-26 02:50:53.865765: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-26 02:50:53.866475: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-26 02:50:53.867265: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [39]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
train['Category'] = encoder.fit_transform(train['class'])

encoding text

In [40]:
# build a vocab_file in the form of numpy array
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
# convert all leters to lowercase
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
#pass the previous two items to the Tokenizer object
tokenizer = bert_tokenization.FullTokenizer(vocab_file, do_lower_case)

def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    for text in texts:
        text = tokenizer.tokenize(text)
        
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len-len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence) + [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
        
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

Building a model
Function we will be using:
Relu
Sofatmax
Cross-entropy:compute cross-entropy loss between real label and predicted label

In [41]:

def build_model(bert_layer, max_len=512):
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")
    
    pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    
    clf_output = sequence_output[:, 0, :]
    lay = tf.keras.layers.Dense(64, activation='relu')(clf_output)
    lay = tf.keras.layers.Dropout(0.2)(lay)
    lay = tf.keras.layers.Dense(32, activation='relu')(lay)
    lay = tf.keras.layers.Dropout(0.2)(lay)
    out = tf.keras.layers.Dense(5, activation='softmax')(lay)
    
    model = tf.keras.models.Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(tf.keras.optimizers.Adam(lr=2e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [42]:
#check each text, we set the length of text as 250 characters
max_len = 250
train_input = bert_encode(train.text.values, tokenizer, max_len=max_len)
test_input = bert_encode(test.text.values, tokenizer, max_len=max_len)
train_labels = y

In [43]:
#define and print labels ,these are character categories s for our submission
labels = label.classes_
print(labels)

['Bart Simpson' 'Homer Simpson' 'Lisa Simpson' 'Marge Simpson' 'Other']


In [44]:
#preveiew model
model = build_model(bert_layer, max_len=max_len)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 250)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 250)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 250)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 109482241   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

/opt/conda/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [45]:
#set parameters and run the model, 10 epochs to increase the accuracy as high as possible 
checkpoint = tf.keras.callbacks.ModelCheckpoint('model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)
train_sh = model.fit(
    train_input, train_labels,
    validation_split=0.2,
    epochs=10,
    callbacks=[checkpoint, earlystopping],
    batch_size=24,
    verbose=1
)



2022-10-26 02:51:27.272892: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10
1622/1622 [==============================] - 1298s 792ms/step - loss: 1.4094 - accuracy: 0.4071 - val_loss: 1.2811 - val_accuracy: 0.4651

Epoch 00001: val_accuracy improved from -inf to 0.46510, saving model to model.h5
Epoch 2/10
1622/1622 [==============================] - 1283s 791ms/step - loss: 1.2646 - accuracy: 0.4819 - val_loss: 1.2489 - val_accuracy: 0.4898

Epoch 00002: val_accuracy improved from 0.46510 to 0.48977, saving model to model.h5
Epoch 3/10
1622/1622 [==============================] - 1283s 791ms/step - loss: 1.1121 - accuracy: 0.5599 - val_loss: 1.3218 - val_accuracy: 0.4836

Epoch 00003: val_accuracy did not improve from 0.48977
Epoch 4/10
1622/1622 [==============================] - 1283s 791ms/step - loss: 0.9203 - accuracy: 0.6500 - val_loss: 1.3200 - val_accuracy: 0.4920

Epoch 00004: val_accuracy improved from 0.48977 to 0.49203, saving model to model.h5
Epoch 5/10
1622/1622 [==============================] - 1283s 791ms/step - loss: 0.7178 - acc

In [46]:
#export prediction as submission.csv
model.load_weights('model.h5')
preds = model.predict(test_input,verbose = 1)
pred_classes = np.argmax(preds, axis = 1)

#mapping the encoded output to actual categories
predicted_category = [labels[x] for x in pred_classes]

submission['Category'] = predicted_category
submission.to_csv('submission.csv', index=False)
display(submission.head(10))

329/329 [==============================] - 105s 318ms/step


,Id,Category
0,1,Bart Simpson
1,2,Lisa Simpson
2,3,Homer Simpson
3,4,Bart Simpson
4,5,Marge Simpson
5,6,Homer Simpson
6,7,Homer Simpson
7,8,Marge Simpson
8,9,Homer Simpson
9,10,Marge Simpson


In [47]:
submission.shape

(10499, 2)